In [1]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler


In [6]:
import pandas as pd

df = pd.read_csv(
    "Avanti_Session01_3.csv",
    skiprows=8,              # skip Delsys metadata, actual data starts after row 8 
    header=None,
    engine="python",         
    on_bad_lines="skip"      # ignore malformed rows
)

# Drop empty columns
df = df.dropna(axis=1, how="all")

df.head()


,0,1,2,3,4,5,6,7,8,9,...,98,99,100,101,102,103,104,105,106,107
0,0.000000,-0.0283665,0,-0.6768799,0,-0.0106201,0,0.7046509,0,-16.641222,...,,,,,,,,,,
1,0.000794,-0.0176242,0.00675,-0.6834717,0.00675,-0.0101929,0.00675,0.704895,0.00675,-16.8091602,...,,,,,,,,,,
2,0.001588,-0.018967,0.0135,-0.6842041,0.0135,-0.0076294,0.0135,0.7036133,0.0135,-16.5648861,...,,,,,,,,,,
3,0.002382,-0.0176242,0.02025,-0.6871338,0.02025,-0.0080566,0.02025,0.697937,0.02025,-16.6641216,...,,,,,,,,,,
4,0.003177,-0.0078889,0.027,-0.6871948,0.027,-0.0090942,0.027,0.6947632,0.027,-16.4274807,...,,,,,,,,,,


In [7]:
# Keep EMG amplitude columns only (every second column)
emg_data = df.iloc[:, 1::2]

# Convert to numeric
emg_data = emg_data.apply(pd.to_numeric, errors="coerce")
emg_data = emg_data.dropna()

emg_data.shape


(0, 54)

In [8]:
import numpy as np
from scipy.signal import butter, filtfilt

fs = 1259  # EMG sampling frequency (Hz)

def bandpass_filter(signal, fs, low=20, high=450, order=4):
    nyq = 0.5 * fs
    b, a = butter(order, [low/nyq, high/nyq], btype='band')
    return filtfilt(b, a, signal)

emg_filtered = emg_data.apply(
    lambda x: bandpass_filter(x.values, fs),
    axis=0
)

In [10]:
# EMG channel names
emg_channels = emg_data.columns

# Label names (one-hot)
labels = labels_df.columns

# Combine signals + labels
emg_complete = pd.concat([emg_data, labels_df], axis=1)

# Insert order index
emg_complete.insert(0, "order", range(len(emg_complete)))

emg_complete.head()


NameError: name 'labels_df' is not defined

In [11]:
def highlight(indices, ax, color):
    i = 0
    while i < len(indices):
        ax.axvspan(
            indices[i] - 0.5,
            indices[i] + 0.5,
            facecolor=color,
            edgecolor='none',
            alpha=0.35
        )
        i += 1


In [12]:
secondsToShow = 5        # EMG looks best in short windows
channelsToShow = 3
labelsToShow = 4

fs = 1259  # EMG sampling frequency

colors = ["red", "purple", "black", "green", "blue", "orange"]
